# Task 2
## Adoption of Mudata to store multimodal data and metadata through Perturb-tools

__|use this jupyter in your home directory|__

## Relevance:
To analyze Perturb-seq experiments is necessary to be able to handle multiple tables with different data and metadata.  
MuData data is a format that allows having all these tables (Guide counts, Transcripts counts, Hashing/multiseq and Metadata)   
in the same data entity and share between different languages:  Python  (https://mudata.readthedocs.io) and R (https://www.bioconductor.org/packages/release/bioc/html/MuData.html).  
Additionally we can provide this MuData  to scripts generating pipeline reports to downstream tasks such as differential Perturbation quantification.

## Practical description: TODO
- [ ] What functionality do we want for the unified data structure?
- [ ] Add  *cis* element selection for a guide, or vice versa. Given a guide select the closest genes ex: 1MB upstream and downstream
- [X] Extract info related to a guide from the MuData object. ex: Given a guide x,  can we extract that guide presence for all cells, extract the counts for each gene being tested  for that guide?
- [X] How to handle negative, positive, controls? Can we subset the MuData object to have only the guides of interest for a given category? How to inject this information in the MuData object?
- [ ] Visualization/object string representation? 
- [ ] Extract each modality for standard scRNA-seq/guide analyses?

## Important links:
Example of class to handle scPERTURB-seq data proposing new operations :
https://colab.research.google.com/drive/1g8oeUkGWOolGc0MeXQ8CseBHwqXHbVPk#scrollTo=iHm0DR23kv4V

In [ ]:
!find  /mnt/shared/gasperini_test_01  | grep raw_mudata

/mnt/shared/gasperini_test_01/93/bc121425eff99125ff0ca316ceb0e7/raw_mudata_guide_and_transcripts.h5mu
/mnt/shared/gasperini_test_01/58/17b340883b2930787a95d103627184/raw_mudata_guide_and_transcripts.h5mu
/mnt/shared/gasperini_test_01/6c/fccd5ca9e34c8034064586053cca0c/raw_mudata_guide_and_transcripts.h5mu
/mnt/shared/gasperini_test_01/3d/400711732bb7fead7f4a800d77456f/raw_mudata_guide_and_transcripts.h5mu


In [ ]:
!pwd

/home/jovyan


In [ ]:
%%writefile mu_mod_workflow_test.nf
params.MUDATA_INPUT = '/mnt/shared/gasperini_test_01/93/bc121425eff99125ff0ca316ceb0e7/raw_mudata_guide_and_transcripts.h5mu'  

process mudata_mod {
    debug true
    input:
    path mudata    
    script:
    """
    chmod 700 /home/jovyan/my_mu_mod.py
    /home/jovyan/my_mu_mod.py ${mudata}
    """
    
}

workflow {
    
    mudata_mod_ou = mudata_mod(params.MUDATA_INPUT)


}

Overwriting mu_mod_workflow_test.nf


In [ ]:
!pwd

/home/jovyan


In [ ]:
%%writefile my_mu_mod.py
#!/usr/bin/env python

import sys
import mudata as md
print ('-----')
print (sys.argv[1])

mudata = md.read(sys.argv[1])



print(mudata['guides'])
print(mudata['scRNA'])

#create a class that can addhandle scPERTURB-SEQ mudata (gudie - scRNA)

Overwriting my_mu_mod.py


In [ ]:
!nextflow run mu_mod_workflow_test.nf -w testing_mu_mod_workflow_test

N E X T F L O W  ~  version 22.10.6
Launching `mu_mod_workflow_test.nf` [desperate_einstein] DSL2 - revision: 675300b63d
[-        ] process > mudata_mod -

executor >  local (1)
[32/876550] process > mudata_mod [  0%] 0 of 1

executor >  local (1)
[32/876550] process > mudata_mod [100%] 1 of 1 ✔
-----
raw_mudata_guide_and_transcripts.h5mu
AnnData object with n_obs × n_vars = 5109 × 98
    obs: 'number_of_nonzero_guides', 'batch_number'
    var: 'feature_name', 'guide_chr', 'guide_end', 'guide_start', 'guide_number', 'target_elements'
AnnData object with n_obs × n_vars = 5109 × 16094
    obs: 'n_genes', 'n_counts', 'percent_mito', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'batch_number'
    var: 'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end'




# Sandbox

In [ ]:
import mudata as md

In [ ]:
mdata = md.read_h5mu("/mnt/shared/gasperini_test_01/93/bc121425eff99125ff0ca316ceb0e7/raw_mudata_guide_and_transcripts.h5mu")

In [ ]:
mdata

In [ ]:
mdata.var

In [ ]:
mdata.var["guides:target_elements"].value_counts()

In [ ]:
# subsetting
mdata[:,mdata.var["guides:target_elements"] == "BROX"]

In [ ]:
# encode gene-guide relationships
